##### Copyright 2023 The TF-Agents Authors.

## Entorno

En Aprendizaje por Refuerzo (RL), un entorno representa la tarea o problema a resolver. Los entornos estándar se pueden crear en TF-Agents utilizando las suites `tf_agents.environments`. TF-Agents tiene suites para cargar entornos de fuentes como OpenAI Gym, Atari y DM Control.

Cargamos el entorno CartPole de la suite OpenAI Gym. Este es el famoso problema del péndulo invertido, donde el objetivo es mantener el poste erguido equilibrando un carro.

Puedes visualizar este entorno para ver cómo se ve. Un poste que se balancea libremente está unido a un carro. El objetivo es mover el carro hacia la derecha o hacia la izquierda para mantener el poste apuntando hacia arriba.

El método `environment.step` toma una `acción` en el entorno y devuelve una tupla `TimeStep` que contiene la siguiente observación del entorno y la recompensa por la acción.

El método `time_step_spec()` devuelve la especificación para la tupla `TimeStep`. Su atributo `observation` muestra la forma de las observaciones, los tipos de datos y los rangos de valores permitidos. El atributo `reward` muestra los mismos detalles para la recompensa.

El método `action_spec()` devuelve la forma, los tipos de datos y los valores permitidos de las acciones válidas.

En el entorno Cartpole:

- `observation` es un array de 4 flotantes:
  - la posición y velocidad del carro
  - la posición angular y velocidad del poste
- `reward` es un valor flotante escalar
- `action` es un valor entero escalar con solo dos posibles valores:
  - `0` — "mover a la izquierda"
  - `1` — "mover a la derecha"

Normalmente se instancian dos entornos: uno para entrenamiento y otro para evaluación.

El entorno Cartpole, como la mayoría de los entornos, está escrito en Python puro. Esto se convierte a TensorFlow usando el wrapper `TFPyEnvironment`.

La API del entorno original utiliza arrays de Numpy. El `TFPyEnvironment` los convierte a `Tensors` para que sea compatible con los agentes y políticas de Tensorflow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Esta celda contiene la licencia de uso del código.

# Train a Deep Q Network with TF-Agents

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Introduction


This example shows how to train a [DQN (Deep Q Networks)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)  agent on the Cartpole environment using the TF-Agents library.

![Cartpole environment](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

It will walk you through all the components in a Reinforcement Learning (RL) pipeline for training, evaluation and data collection.


To run this code live, click the 'Run in Google Colab' link above.


## Setup

If you haven't installed the following dependencies, run:

In [ ]:
# Esta celda instala las dependencias necesarias para ejecutar el cuaderno,
# incluyendo xvfb, ffmpeg, freeglut3-dev para visualización,
# imageio y pyvirtualdisplay para manejar visualizaciones,
# tf-agents[reverb] para el framework de RL y el búfer de replay,
# pyglet para compatibilidad con entornos Gym, y tf-keras para la capa de red neuronal.
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet
!pip install tf-keras

In [ ]:
# Esta celda configura la variable de entorno para asegurar el uso de tf-keras
# en lugar de la nueva versión de Keras (keras-3).
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
# Esta celda importa las librerías necesarias para el tutorial.
# Incluye módulos de Python estándar (__future__),
# librerías para visualización (base64, imageio, IPython, matplotlib, PIL),
# utilidades para visualización virtual (pyvirtualdisplay),
# Reverb para el búfer de replay, TensorFlow y sus submódulos,
# y componentes específicos de TF-Agents para agentes DQN, entornos, políticas, métricas y búferes de replay.
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

In [ ]:
# Esta celda configura una pantalla virtual. Esto es necesario para renderizar
# los entornos de OpenAI Gym en un entorno sin cabeza como Google Colab.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
# Esta celda imprime la versión de TensorFlow que se está utilizando.
tf.version.VERSION

## Hyperparameters

In [ ]:
# Esta celda define los hiperparámetros para el entrenamiento del agente DQN.
# Estos valores controlan el número de iteraciones de entrenamiento,
# la cantidad de pasos iniciales de recolección de datos,
# los pasos de recolección por iteración, el tamaño máximo del búfer de replay,
# el tamaño del lote para el entrenamiento, la tasa de aprendizaje del optimizador,
# el intervalo para registrar el progreso y el intervalo para la evaluación.
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Environment

In Reinforcement Learning (RL), an environment represents the task or problem to be solved. Standard environments can be created in TF-Agents using `tf_agents.environments` suites. TF-Agents has suites for loading environments from sources such as the OpenAI Gym, Atari, and DM Control.

Load the CartPole environment from the OpenAI Gym suite.

In [ ]:
# Esta celda carga el entorno CartPole-v0 desde la suite OpenAI Gym.
# 'env_name' especifica el nombre del entorno.
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

You can render this environment to see how it looks. A free-swinging pole is attached to a cart.  The goal is to move the cart right or left in order to keep the pole pointing up.

In [ ]:
# Esta celda (opcional) renderiza el estado actual del entorno.
# Primero, reinicia el entorno para obtener un estado inicial.
# Luego, utiliza PIL.Image.fromarray(env.render()) para crear una imagen
# del estado actual y la muestra.
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

The `environment.step` method takes an `action` in the environment and returns a `TimeStep` tuple containing the next observation of the environment and the reward for the action.

The `time_step_spec()` method returns the specification for the `TimeStep` tuple. Its `observation` attribute shows the shape of observations, the data types, and the ranges of allowed values. The `reward` attribute shows the same details for the reward.


In [ ]:
# Esta celda imprime la especificación de la observación del entorno.
# Muestra la forma, el tipo de datos y el rango de valores de las observaciones
# que el agente recibirá del entorno.
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
# Esta celda imprime la especificación de la recompensa del entorno.
# Muestra la forma, el tipo de datos y el rango de valores de las recompensas
# que el agente recibirá del entorno.
print('Reward Spec:')
print(env.time_step_spec().reward)

The `action_spec()` method returns the shape, data types, and allowed values of valid actions.

In [ ]:
# Esta celda imprime la especificación de la acción del entorno.
# Muestra la forma, el tipo de datos y los valores permitidos de las acciones
# que el agente puede realizar en el entorno.
print('Action Spec:')
print(env.action_spec())

In the Cartpole environment:

-   `observation` is an array of 4 floats:
    -   the position and velocity of the cart
    -   the angular position and velocity of the pole
-   `reward` is a scalar float value
-   `action` is a scalar integer with only two possible values:
    -   `0` — "move left"
    -   `1` — "move right"


In [ ]:
# Esta celda demuestra cómo interactuar con el entorno.
# Primero, se reinicia el entorno y se obtiene el time_step inicial.
# Luego, se define una acción (en este caso, mover a la derecha con valor 1).
# Finalmente, se ejecuta la acción en el entorno usando env.step() y se imprime
# el siguiente time_step que incluye la nueva observación y la recompensa.
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Usually two environments are instantiated: one for training and one for evaluation.

In [ ]:
# Esta celda instancia dos entornos de CartPole: uno para entrenamiento y otro para evaluación.
# Es una práctica común tener entornos separados para estas dos fases.
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

The Cartpole environment, like most environments, is written in pure Python. This is converted to TensorFlow using the `TFPyEnvironment` wrapper.

The original environment's API uses Numpy arrays. The `TFPyEnvironment` converts these to `Tensors` to make it compatible with Tensorflow agents and policies.


In [ ]:
# Esta celda envuelve los entornos de Python puro (train_py_env y eval_py_env)
# en TFPyEnvironment. Esto convierte las observaciones y acciones de NumPy a Tensors de TensorFlow,
# haciendo que los entornos sean compatibles con los agentes y políticas de TensorFlow.
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agent

The algorithm used to solve an RL problem is represented by an `Agent`. TF-Agents provides standard implementations of a variety of `Agents`, including:

-   [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (used in this tutorial)
-   [REINFORCE](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
-   [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
-   [TD3](https://arxiv.org/pdf/1802.09477.pdf)
-   [PPO](https://arxiv.org/abs/1707.06347)
-   [SAC](https://arxiv.org/abs/1801.01290)

The DQN agent can be used in any environment which has a discrete action space.

At the heart of a DQN Agent is a `QNetwork`, a neural network model that can learn to predict `QValues` (expected returns) for all actions, given an observation from the environment.

We will use `tf_agents.networks.` to create a `QNetwork`. The network will consist of a sequence of `tf.keras.layers.Dense` layers, where the final layer will have 1 output for each possible action.

In [ ]:
# Esta celda define la arquitectura de la red Q (QNetwork) para el agente DQN.
# 'fc_layer_params' define el número de unidades en las capas densas completamente conectadas.
# 'action_tensor_spec' obtiene la especificación del tensor de acción.
# 'num_actions' calcula el número de acciones posibles a partir de la especificación.
# 'dense_layer' es una función de ayuda para crear capas densas con la configuración adecuada (activación ReLU e inicializador de kernel).
# 'dense_layers' crea la secuencia de capas densas ocultas.
# 'q_values_layer' es la capa de salida que produce un valor Q para cada acción posible.
# 'q_net' es el modelo Sequential que combina las capas densas y la capa de salida.
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

Now use `tf_agents.agents.dqn.dqn_agent` to instantiate a `DqnAgent`. In addition to the `time_step_spec`, `action_spec` and the QNetwork, the agent constructor also requires an optimizer (in this case, `AdamOptimizer`), a loss function, and an integer step counter.

In [ ]:
# Esta celda instancia el agente DQN.
# Se le pasan las especificaciones del time_step y la acción del entorno,
# la red Q ('q_net') que definimos, un optimizador (Adam),
# la función de pérdida para los errores temporales diferenciales (TD)
# y un contador de pasos de entrenamiento.
# Finalmente, se inicializa el agente.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Policies

A policy defines the way an agent acts in an environment. Typically, the goal of reinforcement learning is to train the underlying model until the policy produces the desired outcome.

In this tutorial:

-   The desired outcome is keeping the pole balanced upright over the cart.
-   The policy returns an action (left or right) for each `time_step` observation.

Agents contain two policies:

-   `agent.policy` — The main policy that is used for evaluation and deployment.
-   `agent.collect_policy` — A second policy that is used for data collection.


In [ ]:
# Esta celda extrae las políticas de evaluación y recolección del agente.
# 'eval_policy' se usa para evaluar el rendimiento del agente sin exploración.
# 'collect_policy' se usa para recolectar datos del entorno, a menudo con alguna forma de exploración (como epsilon-greedy en DQN).
eval_policy = agent.policy
collect_policy = agent.collect_policy

Policies can be created independently of agents. For example, use `tf_agents.policies.random_tf_policy` to create a policy which will randomly select an action for each `time_step`.

In [ ]:
# Esta celda crea una política aleatoria.
# Esta política selecciona acciones al azar y se utiliza a menudo como línea base
# para comparar el rendimiento del agente entrenado.
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

To get an action from a policy, call the `policy.action(time_step)` method. The `time_step` contains the observation from the environment. This method returns a `PolicyStep`, which is a named tuple with three components:

-   `action` — the action to be taken (in this case, `0` or `1`)
-   `state` — used for stateful (that is, RNN-based) policies
-   `info` — auxiliary data, such as log probabilities of actions

In [ ]:
# Esta celda crea una instancia de un entorno de ejemplo envuelto en TFPyEnvironment
# para demostrar cómo obtener una acción de una política.
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [ ]:
# Esta celda reinicia el entorno de ejemplo para obtener un time_step inicial.
time_step = example_environment.reset()

In [ ]:
# Esta celda llama al método .action() de la política aleatoria
# con el time_step actual para obtener una acción.
# Imprime el PolicyStep resultante, que contiene la acción seleccionada.
random_policy.action(time_step)

## Metrics and Evaluation

The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode. Several episodes are run, creating an average return.

The following function computes the average return of a policy, given the policy, environment, and a number of episodes.


In [ ]:
# Esta celda define una función para calcular el retorno promedio de una política.
# Toma el entorno, la política y el número de episodios como entrada.
# Simula la ejecución de la política en el entorno durante el número especificado de episodios,
# suma las recompensas obtenidas en cada episodio y devuelve el promedio.
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Running this computation on the `random_policy` shows a baseline performance in the environment.

In [ ]:
# Esta celda calcula e imprime el retorno promedio de la política aleatoria
# utilizando la función compute_avg_return. Esto sirve como línea base de rendimiento.
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Replay Buffer

In order to keep track of the data collected from the environment, we will use [Reverb](https://deepmind.com/research/open-source/Reverb), an efficient, extensible, and easy-to-use replay system by Deepmind. It stores experience data when we collect trajectories and is consumed during training.

This replay buffer is constructed using specs describing the tensors that are to be stored, which can be obtained from the agent using agent.collect_data_spec.


In [ ]:
# Esta celda configura el búfer de replay utilizando Reverb.
# Se define una tabla con nombre ('uniform_table') con un tamaño máximo.
# Se utilizan selectores Uniform (para muestreo aleatorio) y Fifo (para eliminación).
# Se crea un servidor Reverb local para gestionar la tabla.
# Se instancia ReverbReplayBuffer para interactuar con el servidor Reverb.
# Finalmente, se crea un observador (rb_observer) que añadirá trayectorias
# al búfer de replay cuando se ejecute el driver de recolección.
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

In [ ]:
# Esta celda imprime la especificación de datos de recolección del agente.
# Muestra la estructura de los datos (Trayectoria) que se almacenarán
# en el búfer de replay.
agent.collect_data_spec

In [ ]:
# Esta celda imprime los nombres de los campos dentro de la especificación
# de datos de recolección del agente (Trajectory).
agent.collect_data_spec._fields

## Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

Here we are using 'PyDriver' to run the experience collecting loop. You can learn more about TF Agents driver in our [drivers tutorial](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
# Esta celda ejecuta el driver de recolección de datos inicial.
# Utiliza PyDriver para interactuar con el entorno de Python.
# Ejecuta la política aleatoria ('random_policy') durante 'initial_collect_steps' pasos.
# El observador 'rb_observer' registra las trayectorias recopiladas en el búfer de replay.
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

The replay buffer is now a collection of Trajectories.

In [ ]:
# Esta celda (comentada) muestra cómo inspeccionar un elemento individual
# del búfer de replay iterando sobre su conjunto de datos.
# # For the curious:
# # Uncomment to peel one of these off and inspect it.
# # iter(replay_buffer.as_dataset()).next()

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

This dataset is also optimized by running parallel calls and prefetching data.

In [ ]:
# Esta celda crea un conjunto de datos ('dataset') a partir del búfer de replay.
# Este conjunto de datos se utilizará para alimentar los datos de entrenamiento al agente.
# Se configura para muestrear lotes de datos con un tamaño de lote especificado,
# y para incluir secuencias de 2 pasos (observación actual y siguiente)
# que son necesarias para calcular la pérdida de DQN.
# Se optimiza con llamadas paralelas y prefetching para mejorar el rendimiento.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:
# Esta celda crea un iterador para el conjunto de datos.
# Este iterador se utilizará en el bucle de entrenamiento para obtener lotes de datos.
iterator = iter(dataset)
print(iterator)

In [ ]:
# Esta celda (comentada) muestra cómo obtener el siguiente lote de datos
# del iterador del conjunto de datos para inspeccionarlo.
# # For the curious:
# # Uncomment to see what the dataset iterator is feeding to the agent.
# # Compare this representation of replay data
# # to the collection of individual trajectories shown earlier.

# # iterator.next()

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [ ]:
# Esta celda contiene el bucle principal de entrenamiento del agente.
# Primero, se mide el tiempo de ejecución (opcional).
# Se envuelve la función de entrenamiento del agente con tf.function para optimizarla.
# Se reinicia el contador de pasos de entrenamiento.
# Se evalúa la política del agente antes de comenzar el entrenamiento para tener una línea base.
# Se reinicia el entorno de entrenamiento.
# Se crea un driver de recolección para la política del agente.
# El bucle principal itera 'num_iterations' veces:
# 1. Recopila datos del entorno utilizando el driver y la política de recolección del agente.
# 2. Muestra un lote de datos del búfer de replay utilizando el iterador.
# 3. Entrena al agente utilizando el lote de experiencia y calcula la pérdida.
# 4. Registra la pérdida periódicamente (cada 'log_interval' pasos).
# 5. Evalúa la política del agente periódicamente (cada 'eval_interval' pasos)
#    y almacena el retorno promedio.
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

In [ ]:
# Esta celda genera un gráfico que muestra cómo el retorno promedio de la política
# mejoró a lo largo de las iteraciones de entrenamiento.
# 'iterations' crea una lista de los pasos en los que se realizó la evaluación.
# 'plt.plot' dibuja la línea del retorno promedio frente a las iteraciones.
# Se etiquetan los ejes y se establece un límite superior para el eje y.
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Videos

Charts are nice. But more exciting is seeing an agent actually performing a task in an environment.

First, create a function to embed videos in the notebook.

In [ ]:
# Esta celda define una función de ayuda para incrustar un archivo de video MP4
# directamente en la salida del cuaderno de Colab.
# Lee el archivo, lo codifica en base64 y genera una etiqueta HTML de video.
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Now iterate through a few episodes of the Cartpole game with the agent. The underlying Python environment (the one "inside" the TensorFlow environment wrapper) provides a `render()` method, which outputs an image of the environment state. These can be collected into a video.

In [ ]:
# Esta celda define una función para crear un video de la evaluación de una política.
# Toma la política, el nombre del archivo de salida y el número de episodios como entrada.
# Utiliza imageio para escribir los fotogramas del entorno renderizado en un archivo MP4.
# Simula la ejecución de la política en el entorno de evaluación,
# renderiza cada paso de tiempo y añade el fotograma al video.
# Finalmente, llama a la función embed_mp4 para mostrar el video en el cuaderno.
create_policy_eval_video(agent.policy, "trained-agent")

For fun, compare the trained agent (above) to an agent moving randomly. (It does not do as well.)

In [ ]:
# Esta celda crea un video de la política aleatoria para comparar su rendimiento
# con el agente entrenado. Se espera que el agente aleatorio tenga un rendimiento mucho peor.
create_policy_eval_video(random_policy, "random-agent")